This is a notebook for pulling down the Imdb data using the Omdb API. 

In [1]:
import pandas as pd
import requests
import re
import json
import requests
import time

Read in the movies.csv file that we scraped from <website> using simple copy and paste into Microsoft Excel.

In [23]:
!ls

allTime.csv
clean_movies_01.csv
movies.csv
omdb_extract.ipynb
README.md
Untitled.ipynb
Untitled1.ipynb


In [24]:
df = pd.read_csv('allTime.csv')
df.head()

,Rank,Title,Worldwide Lifetime Gross,Domestic Lifetime Gross,Domestic %,Foreign Lifetime Gross,Foreign %,Year
0,1,Avengers: Endgame,"$2,797,800,564","$858,373,000",30.70%,"$1,939,427,564",69.30%,2019
1,2,Avatar,"$2,789,968,301","$760,507,625",27.30%,"$2,029,460,676",72.70%,2009
2,3,Titanic,"$2,187,463,944","$659,363,944",30.10%,"$1,528,100,000",69.90%,1997
3,4,Star Wars: Episode VII - The Force Awakens,"$2,068,223,624","$936,662,225",45.30%,"$1,131,561,399",54.70%,2015
4,5,Avengers: Infinity War,"$2,048,359,754","$678,815,482",33.10%,"$1,369,544,272",66.90%,2018


In [4]:
format(ord(";"), "x")

'3b'

In [3]:
def reformat_movie_name(name):
    special_chars = { ' ': "+",   ':': "%3A", '.' : "%2E",
                      '!': "%21", '-': '%2D', '\'': "%27", 
                      '?': '$3F', '&': "%26", '/': "%2F",
                      ',': "%2C", ';': "%3B" 
                     }

    reformatted = ""
    for c in name:
        if c in special_chars:
           reformatted += special_chars[c]
        else:
           reformatted += c
    return reformatted

In [5]:
reformat_movie_name("Avengers: Endgame")

'Avengers%3A+Endgame'

In [7]:
# Need to import json to do this


# Define a function to open the json
def get_keys(path):
    with open(path) as f:
        return json.load(f)

# Using the function to open and load all keys in that file 
api_keys = get_keys("/Users/anupj/Documents/APIs/omdb.json")

# Setting the first (and only) value as a variable
key = list(api_keys.values())[0]

In [171]:
key=get_key("/Users/frkornet/secrets/omdb_api_key.json")

Test out the request on a single movie to make sure it works, so we do not run out of requests for the day.

In [8]:
movie="Avengers: Endgame"
movie = reformat_movie_name(movie)
movie_request = f"http://www.omdbapi.com/?t={movie}&apikey={key}"
print(movie_request)
resp = requests.get(movie_request)

http://www.omdbapi.com/?t=Avengers%3A+Endgame&apikey=b19a7eb8


In [9]:
print(resp)

<Response [200]>


In [10]:
print(resp.json())

{'Title': 'Avengers: Endgame', 'Year': '2019', 'Rated': 'PG-13', 'Released': '26 Apr 2019', 'Runtime': '181 min', 'Genre': 'Action, Adventure, Drama, Sci-Fi', 'Director': 'Anthony Russo, Joe Russo', 'Writer': 'Christopher Markus (screenplay by), Stephen McFeely (screenplay by), Stan Lee (based on the Marvel comics by), Jack Kirby (based on the Marvel comics by), Joe Simon (Captain America created by), Jack Kirby (Captain America created by), Steve Englehart (Star-Lord created by), Steve Gan (Star-Lord created by), Bill Mantlo (Rocket Raccoon created by), Keith Giffen (Rocket Raccoon created by), Jim Starlin (Thanos,  Gamora & Drax created by), Stan Lee (Groot created by), Larry Lieber (Groot created by), Jack Kirby (Groot created by), Steve Englehart (Mantis created by), Don Heck (Mantis created by)', 'Actors': 'Robert Downey Jr., Chris Evans, Mark Ruffalo, Chris Hemsworth', 'Plot': "After the devastating events of Avengers: Infinity War (2018), the universe is in ruins. With the help 

Surprisingly the data frame created from the json dictionary contains three rows!

In [11]:
movie_df = pd.DataFrame(resp.json())
movie_df

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,Metascore,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response
0,Avengers: Endgame,2019,PG-13,26 Apr 2019,181 min,"Action, Adventure, Drama, Sci-Fi","Anthony Russo, Joe Russo","Christopher Markus (screenplay by), Stephen Mc...","Robert Downey Jr., Chris Evans, Mark Ruffalo, ...",After the devastating events of Avengers: Infi...,...,78,8.5,"589,503",tt4154796,movie,30 Jul 2019,N/A,Marvel Studios,N/A,True
1,Avengers: Endgame,2019,PG-13,26 Apr 2019,181 min,"Action, Adventure, Drama, Sci-Fi","Anthony Russo, Joe Russo","Christopher Markus (screenplay by), Stephen Mc...","Robert Downey Jr., Chris Evans, Mark Ruffalo, ...",After the devastating events of Avengers: Infi...,...,78,8.5,"589,503",tt4154796,movie,30 Jul 2019,N/A,Marvel Studios,N/A,True
2,Avengers: Endgame,2019,PG-13,26 Apr 2019,181 min,"Action, Adventure, Drama, Sci-Fi","Anthony Russo, Joe Russo","Christopher Markus (screenplay by), Stephen Mc...","Robert Downey Jr., Chris Evans, Mark Ruffalo, ...",After the devastating events of Avengers: Infi...,...,78,8.5,"589,503",tt4154796,movie,30 Jul 2019,N/A,Marvel Studios,N/A,True


In [12]:
movie_df.columns

Index(['Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director',
       'Writer', 'Actors', 'Plot', 'Language', 'Country', 'Awards', 'Poster',
       'Ratings', 'Metascore', 'imdbRating', 'imdbVotes', 'imdbID', 'Type',
       'DVD', 'BoxOffice', 'Production', 'Website', 'Response'],
      dtype='object')

The following code snippet checks whether all the rows are the same. It checks all possible combinations of the rows for each column. If it is not the same it prints the column name. As you can see the Ratings column is different for the various rows.

In [13]:
bool_list = []
for col in movie_df.columns:
    if movie_df[col][0] != movie_df[col][1]:
       print(f"Column {col}: row 0 and row 1 not equal!")
    bool_list.append(movie_df[col][0] == movie_df[col][1])
    
    if movie_df[col][0] != movie_df[col][2]:
       print(f"Column {col}: row 0 and row 2 not equal!")
    bool_list.append(movie_df[col][0] == movie_df[col][2])
    
    if movie_df[col][1] != movie_df[col][2]:
       print(f"Column {col}: row 1 and row 2 not equal!")
    bool_list.append(movie_df[col][1] == movie_df[col][2])

all(bool_list)

Column Ratings: row 0 and row 1 not equal!
Column Ratings: row 0 and row 2 not equal!
Column Ratings: row 1 and row 2 not equal!


False

In [14]:
list(movie_df['Ratings'])

[{'Source': 'Internet Movie Database', 'Value': '8.5/10'},
 {'Source': 'Rotten Tomatoes', 'Value': '94%'},
 {'Source': 'Metacritic', 'Value': '78/100'}]

In [15]:
col_set = set()
for col in movie_df.columns:
    if col != 'Ratings':
       print(col)
       col_set.add(col)     
print(col_set)

Title
Year
Rated
Released
Runtime
Genre
Director
Writer
Actors
Plot
Language
Country
Awards
Poster
Metascore
imdbRating
imdbVotes
imdbID
Type
DVD
BoxOffice
Production
Website
Response
{'Actors', 'Plot', 'BoxOffice', 'Country', 'Awards', 'Genre', 'Title', 'Runtime', 'Released', 'Response', 'Director', 'Poster', 'Year', 'Type', 'imdbRating', 'Writer', 'imdbVotes', 'Rated', 'Production', 'Metascore', 'Language', 'imdbID', 'DVD', 'Website'}


In [16]:
movie_wo_ratings = movie_df[list(col_set)].iloc[0]

In [17]:
movie_wo_ratings['Ratings'] = list(movie_df['Ratings'])

In [18]:
movie_wo_ratings

Actors        Robert Downey Jr., Chris Evans, Mark Ruffalo, ...
Plot          After the devastating events of Avengers: Infi...
BoxOffice                                                   N/A
Country                                                     USA
Awards                                                      N/A
Genre                          Action, Adventure, Drama, Sci-Fi
Title                                         Avengers: Endgame
Runtime                                                 181 min
Released                                            26 Apr 2019
Response                                                   True
Director                               Anthony Russo, Joe Russo
Poster        https://m.media-amazon.com/images/M/MV5BMTc5MD...
Year                                                       2019
Type                                                      movie
imdbRating                                                  8.5
Writer        Christopher Markus (screen

In [19]:
all_movies_df = pd.DataFrame(columns=movie_df.columns)
all_movies_df

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,Metascore,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response


In [20]:
all_movies_df = all_movies_df.append(movie_wo_ratings)

In [21]:
all_movies_df

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,Metascore,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response
0,Avengers: Endgame,2019,PG-13,26 Apr 2019,181 min,"Action, Adventure, Drama, Sci-Fi","Anthony Russo, Joe Russo","Christopher Markus (screenplay by), Stephen Mc...","Robert Downey Jr., Chris Evans, Mark Ruffalo, ...",After the devastating events of Avengers: Infi...,...,78,8.5,"589,503",tt4154796,movie,30 Jul 2019,N/A,Marvel Studios,N/A,True


In [22]:
all_movies_df = all_movies_df.append(movie_wo_ratings)
all_movies_df

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,Metascore,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response
0,Avengers: Endgame,2019,PG-13,26 Apr 2019,181 min,"Action, Adventure, Drama, Sci-Fi","Anthony Russo, Joe Russo","Christopher Markus (screenplay by), Stephen Mc...","Robert Downey Jr., Chris Evans, Mark Ruffalo, ...",After the devastating events of Avengers: Infi...,...,78,8.5,"589,503",tt4154796,movie,30 Jul 2019,N/A,Marvel Studios,N/A,True
0,Avengers: Endgame,2019,PG-13,26 Apr 2019,181 min,"Action, Adventure, Drama, Sci-Fi","Anthony Russo, Joe Russo","Christopher Markus (screenplay by), Stephen Mc...","Robert Downey Jr., Chris Evans, Mark Ruffalo, ...",After the devastating events of Avengers: Infi...,...,78,8.5,"589,503",tt4154796,movie,30 Jul 2019,N/A,Marvel Studios,N/A,True


In [25]:
def get_movie(movie):
    print(movie)
    movie = reformat_movie_name(movie)
    print(movie)
    movie_request = f"http://www.omdbapi.com/?t={movie}&apikey={key}"
    print(movie_request)

    resp = requests.get(movie_request)

    movie_df = pd.DataFrame(resp.json())
#    print(movie_df)
    
    col_set = set()
    for col in movie_df.columns:
        if col != 'Ratings':
           col_set.add(col)
    
    cleaned_movie = movie_df[list(col_set)].iloc[0]
    cleaned_movie['Ratings'] = list(movie_df['Ratings'])
    return cleaned_movie

In [26]:
cleaned_movie = get_movie("Avengers: Endgame")
cleaned_movie

Avengers: Endgame
Avengers%3A+Endgame
http://www.omdbapi.com/?t=Avengers%3A+Endgame&apikey=b19a7eb8


Actors        Robert Downey Jr., Chris Evans, Mark Ruffalo, ...
Plot          After the devastating events of Avengers: Infi...
BoxOffice                                                   N/A
Country                                                     USA
Awards                                                      N/A
Genre                          Action, Adventure, Drama, Sci-Fi
Title                                         Avengers: Endgame
Runtime                                                 181 min
Released                                            26 Apr 2019
Response                                                   True
Director                               Anthony Russo, Joe Russo
Poster        https://m.media-amazon.com/images/M/MV5BMTc5MD...
Year                                                       2019
Type                                                      movie
imdbRating                                                  8.5
Writer        Christopher Markus (screen

In [27]:
all_movies_df = pd.DataFrame(columns=movie_df.columns)
all_movies_df

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,Metascore,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response


In [28]:
error_list = []

In [29]:
for i, movie in enumerate(df['Title']):
        
    print(f"Getting movie {i} {movie}")
    try:
       cleaned_movie = get_movie(movie)
       all_movies_df = all_movies_df.append(cleaned_movie)
       time.sleep(1)
    except:
       print("Error retrieving movie {i} {movie}")
       error_list.append(i+1)

all_movies_df.head()

Getting movie 0 Avengers: Endgame
Avengers: Endgame
Avengers%3A+Endgame
http://www.omdbapi.com/?t=Avengers%3A+Endgame&apikey=b19a7eb8
Getting movie 1 Avatar
Avatar
Avatar
http://www.omdbapi.com/?t=Avatar&apikey=b19a7eb8
Getting movie 2 Titanic
Titanic
Titanic
http://www.omdbapi.com/?t=Titanic&apikey=b19a7eb8
Getting movie 3 Star Wars: Episode VII - The Force Awakens
Star Wars: Episode VII - The Force Awakens
Star+Wars%3A+Episode+VII+%2D+The+Force+Awakens
http://www.omdbapi.com/?t=Star+Wars%3A+Episode+VII+%2D+The+Force+Awakens&apikey=b19a7eb8
Getting movie 4 Avengers: Infinity War
Avengers: Infinity War
Avengers%3A+Infinity+War
http://www.omdbapi.com/?t=Avengers%3A+Infinity+War&apikey=b19a7eb8
Getting movie 5 Jurassic World
Jurassic World
Jurassic+World
http://www.omdbapi.com/?t=Jurassic+World&apikey=b19a7eb8
Getting movie 6 The Lion King
The Lion King
The+Lion+King
http://www.omdbapi.com/?t=The+Lion+King&apikey=b19a7eb8
Getting movie 7 The Avengers
The Avengers
The+Avengers
http://www.

Getting movie 55 Harry Potter and the Half-Blood Prince
Harry Potter and the Half-Blood Prince
Harry+Potter+and+the+Half%2DBlood+Prince
http://www.omdbapi.com/?t=Harry+Potter+and+the+Half%2DBlood+Prince&apikey=b19a7eb8
Getting movie 56 The Lord of the Rings: The Two Towers
The Lord of the Rings: The Two Towers
The+Lord+of+the+Rings%3A+The+Two+Towers
http://www.omdbapi.com/?t=The+Lord+of+the+Rings%3A+The+Two+Towers&apikey=b19a7eb8
Getting movie 57 Shrek 2
Shrek 2
Shrek+2
http://www.omdbapi.com/?t=Shrek+2&apikey=b19a7eb8
Getting movie 58 Bohemian Rhapsody
Bohemian Rhapsody
Bohemian+Rhapsody
http://www.omdbapi.com/?t=Bohemian+Rhapsody&apikey=b19a7eb8
Getting movie 59 Harry Potter and the Goblet of Fire
Harry Potter and the Goblet of Fire
Harry+Potter+and+the+Goblet+of+Fire
http://www.omdbapi.com/?t=Harry+Potter+and+the+Goblet+of+Fire&apikey=b19a7eb8
Getting movie 60 Spider-Man 3
Spider-Man 3
Spider%2DMan+3
http://www.omdbapi.com/?t=Spider%2DMan+3&apikey=b19a7eb8
Getting movie 61 Ice Age: 

Getting movie 107 Frozen II
Frozen II
Frozen+II
http://www.omdbapi.com/?t=Frozen+II&apikey=b19a7eb8
Getting movie 108 The Chronicles of Narnia: The Lion, the Witch and the Wardrobe
The Chronicles of Narnia: The Lion, the Witch and the Wardrobe
The+Chronicles+of+Narnia%3A+The+Lion%2C+the+Witch+and+the+Wardrobe
http://www.omdbapi.com/?t=The+Chronicles+of+Narnia%3A+The+Lion%2C+the+Witch+and+the+Wardrobe&apikey=b19a7eb8
Getting movie 109 Monsters University
Monsters University
Monsters+University
http://www.omdbapi.com/?t=Monsters+University&apikey=b19a7eb8
Getting movie 110 Up
Up
Up
http://www.omdbapi.com/?t=Up&apikey=b19a7eb8
Getting movie 111 Gravity
Gravity
Gravity
http://www.omdbapi.com/?t=Gravity&apikey=b19a7eb8
Getting movie 112 Captain America: The Winter Soldier
Captain America: The Winter Soldier
Captain+America%3A+The+Winter+Soldier
http://www.omdbapi.com/?t=Captain+America%3A+The+Winter+Soldier&apikey=b19a7eb8
Getting movie 113 The Twilight Saga: Breaking Dawn - Part 1
The Twil

Getting movie 164 The Hangover Part II
The Hangover Part II
The+Hangover+Part+II
http://www.omdbapi.com/?t=The+Hangover+Part+II&apikey=b19a7eb8
Getting movie 165 Quantum of Solace
Quantum of Solace
Quantum+of+Solace
http://www.omdbapi.com/?t=Quantum+of+Solace&apikey=b19a7eb8
Getting movie 166 Iron Man
Iron Man
Iron+Man
http://www.omdbapi.com/?t=Iron+Man&apikey=b19a7eb8
Getting movie 167 I Am Legend
I Am Legend
I+Am+Legend
http://www.omdbapi.com/?t=I+Am+Legend&apikey=b19a7eb8
Getting movie 168 Ready Player One
Ready Player One
Ready+Player+One
http://www.omdbapi.com/?t=Ready+Player+One&apikey=b19a7eb8
Getting movie 169 Hong hai xing dong
Hong hai xing dong
Hong+hai+xing+dong
http://www.omdbapi.com/?t=Hong+hai+xing+dong&apikey=b19a7eb8
Error retrieving movie {i} {movie}
Getting movie 170 Night at the Museum
Night at the Museum
Night+at+the+Museum
http://www.omdbapi.com/?t=Night+at+the+Museum&apikey=b19a7eb8
Getting movie 171 Fifty Shades of Grey
Fifty Shades of Grey
Fifty+Shades+of+Grey


,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,Metascore,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response
0,Avengers: Endgame,2019,PG-13,26 Apr 2019,181 min,"Action, Adventure, Drama, Sci-Fi","Anthony Russo, Joe Russo","Christopher Markus (screenplay by), Stephen Mc...","Robert Downey Jr., Chris Evans, Mark Ruffalo, ...",After the devastating events of Avengers: Infi...,...,78,8.5,"589,503",tt4154796,movie,30 Jul 2019,N/A,Marvel Studios,N/A,True
0,Avatar,2009,PG-13,18 Dec 2009,162 min,"Action, Adventure, Fantasy, Sci-Fi",James Cameron,James Cameron,"Sam Worthington, Zoe Saldana, Sigourney Weaver...",A paraplegic Marine dispatched to the moon Pan...,...,83,7.8,"1,064,516",tt0499549,movie,22 Apr 2010,"$749,700,000",20th Century Fox,N/A,True
0,Titanic,1997,PG-13,19 Dec 1997,194 min,"Drama, Romance",James Cameron,James Cameron,"Leonardo DiCaprio, Kate Winslet, Billy Zane, K...",A seventeen-year-old aristocrat falls in love ...,...,75,7.8,"978,848",tt0120338,movie,10 Sep 2012,N/A,Paramount Pictures,N/A,True
0,Star Wars: Episode VII - The Force Awakens,2015,PG-13,18 Dec 2015,138 min,"Action, Adventure, Sci-Fi",J.J. Abrams,"Lawrence Kasdan, J.J. Abrams, Michael Arndt, G...","Harrison Ford, Mark Hamill, Carrie Fisher, Ada...","Three decades after the Empire's defeat, a new...",...,81,8.0,"797,388",tt2488496,movie,05 Apr 2016,"$936,658,640",Walt Disney Pictures,N/A,True
0,Avengers: Infinity War,2018,PG-13,27 Apr 2018,149 min,"Action, Adventure, Sci-Fi","Anthony Russo, Joe Russo","Christopher Markus (screenplay by), Stephen Mc...","Robert Downey Jr., Chris Hemsworth, Mark Ruffa...",The Avengers and their allies must be willing ...,...,68,8.5,"725,561",tt4154756,movie,14 Aug 2018,"$664,987,816",Walt Disney Pictures,N/A,True


In [30]:
all_movies_df


,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,Metascore,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response
0,Avengers: Endgame,2019,PG-13,26 Apr 2019,181 min,"Action, Adventure, Drama, Sci-Fi","Anthony Russo, Joe Russo","Christopher Markus (screenplay by), Stephen Mc...","Robert Downey Jr., Chris Evans, Mark Ruffalo, ...",After the devastating events of Avengers: Infi...,...,78,8.5,"589,503",tt4154796,movie,30 Jul 2019,N/A,Marvel Studios,N/A,True
0,Avatar,2009,PG-13,18 Dec 2009,162 min,"Action, Adventure, Fantasy, Sci-Fi",James Cameron,James Cameron,"Sam Worthington, Zoe Saldana, Sigourney Weaver...",A paraplegic Marine dispatched to the moon Pan...,...,83,7.8,"1,064,516",tt0499549,movie,22 Apr 2010,"$749,700,000",20th Century Fox,N/A,True
0,Titanic,1997,PG-13,19 Dec 1997,194 min,"Drama, Romance",James Cameron,James Cameron,"Leonardo DiCaprio, Kate Winslet, Billy Zane, K...",A seventeen-year-old aristocrat falls in love ...,...,75,7.8,"978,848",tt0120338,movie,10 Sep 2012,N/A,Paramount Pictures,N/A,True
0,Star Wars: Episode VII - The Force Awakens,2015,PG-13,18 Dec 2015,138 min,"Action, Adventure, Sci-Fi",J.J. Abrams,"Lawrence Kasdan, J.J. Abrams, Michael Arndt, G...","Harrison Ford, Mark Hamill, Carrie Fisher, Ada...","Three decades after the Empire's defeat, a new...",...,81,8.0,"797,388",tt2488496,movie,05 Apr 2016,"$936,658,640",Walt Disney Pictures,N/A,True
0,Avengers: Infinity War,2018,PG-13,27 Apr 2018,149 min,"Action, Adventure, Sci-Fi","Anthony Russo, Joe Russo","Christopher Markus (screenplay by), Stephen Mc...","Robert Downey Jr., Chris Hemsworth, Mark Ruffa...",The Avengers and their allies must be willing ...,...,68,8.5,"725,561",tt4154756,movie,14 Aug 2018,"$664,987,816",Walt Disney Pictures,N/A,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Hotel Transylvania 3: Summer Vacation,2018,PG,13 Jul 2018,97 min,"Animation, Adventure, Comedy, Family, Fantasy",Genndy Tartakovsky,"Genndy Tartakovsky, Michael McCullers","Adam Sandler, Andy Samberg, Selena Gomez, Kevi...",Count Dracula and company participate in a cru...,...,54,6.3,"46,865",tt5220122,movie,25 Sep 2018,N/A,Columbia Pictures / Sony Pictures Animation,N/A,True
0,The Boss Baby,2017,PG,31 Mar 2017,97 min,"Animation, Adventure, Comedy, Family, Fantasy",Tom McGrath,"Michael McCullers, Marla Frazee (based on the ...","Alec Baldwin, Steve Buscemi, Jimmy Kimmel, Lis...","A suit-wearing, briefcase-carrying baby pairs ...",...,50,6.3,"97,604",tt3874544,movie,25 Jul 2017,"$174,996,080",DreamWorks Animation,N/A,True
0,The Simpsons Movie,2007,PG-13,27 Jul 2007,87 min,"Animation, Adventure, Comedy",David Silverman,"James L. Brooks (screenplay), Matt Groening (s...","Dan Castellaneta, Julie Kavner, Nancy Cartwrig...",After Homer accidentally pollutes the town's w...,...,80,7.3,"296,841",tt0462538,movie,18 Dec 2007,"$183,100,000",20th Century Fox,N/A,True
0,Dunkirk,2017,PG-13,21 Jul 2017,106 min,"Action, Drama, History, Thriller, War",Christopher Nolan,Christopher Nolan,"Fionn Whitehead, Damien Bonnard, Aneurin Barna...","Allied soldiers from Belgium, the British Empi...",...,94,7.9,"486,426",tt5013056,movie,19 Dec 2017,"$188,042,171",Warner Bros. Pictures,N/A,True


In [31]:
all_movies_df.shape

(193, 25)

In [251]:
error_list

In [250]:
df['Release Group'][170:180]

170                       Arctic Dogs
171                  Official Secrets
172                      Run the Race
173                       The Mustang
174                     Immortal Hero
175    National Theatre Live: Fleabag
176                   Cyrano, My Love
177      The Professor and the Madman
178                             Furie
179                             Saaho
Name: Release Group, dtype: object

In [272]:
error_list = sorted(error_list)

In [334]:
all_movies_df[ all_movies_df["Title"] == "Kiki's Delivery Service"]

,Actors,Awards,BoxOffice,Country,DVD,Director,Genre,Language,Metascore,Plot,...,Type,Website,Writer,Year,imdbID,imdbRating,imdbVotes,index,level_0,totalSeasons
0,"Minami Takayama, Rei Sakuma, Kappei Yamaguchi,...",4 wins.,N/A,Japan,08 May 2007,Hayao Miyazaki,"Animation, Adventure, Drama, Family, Fantasy","Japanese, English",83,"A young witch, on her mandatory year of indepe...",...,movie,N/A,"Eiko Kadono (novel), Hayao Miyazaki (screenplay)",1989,tt0097814,7.9,"102,275",NaN,NaN,NaN
0,"Minami Takayama, Rei Sakuma, Kappei Yamaguchi,...",4 wins.,N/A,Japan,08 May 2007,Hayao Miyazaki,"Animation, Adventure, Drama, Family, Fantasy","Japanese, English",83,"A young witch, on her mandatory year of indepe...",...,movie,N/A,"Eiko Kadono (novel), Hayao Miyazaki (screenplay)",1989,tt0097814,7.9,"102,275",NaN,NaN,NaN


In [346]:
all_movies_df[ all_movies_df["Title"] == "Fantastic Fungi"]

,Actors,Awards,BoxOffice,Country,DVD,Director,Genre,Language,Metascore,Plot,...,Type,Website,Writer,Year,imdbID,imdbRating,imdbVotes,index,level_0,totalSeasons
0,"Tony D. Head, Michael Pollan, Paul Stamets, An...",N/A,N/A,USA,N/A,Louie Schwartzberg,Documentary,English,N/A,"June 12, 2019 From the Maui Film Festival's Ce...",...,movie,N/A,Mark Monroe,2018,tt8258074,N/A,N/A,NaN,NaN,NaN


In [345]:
a = {"Title":"Fantastic Fungi","Year":"2018","Rated":"N/A","Released":"N/A","Runtime":"N/A","Genre":"Documentary","Director":"Louie Schwartzberg","Writer":"Mark Monroe","Actors":"Tony D. Head, Michael Pollan, Paul Stamets, Andrew Weil","Plot":"June 12, 2019 From the Maui Film Festival's Celestial Cinema. Imagine an organism that feeds you, heals you, reveals secrets of the universe and could help save the planet. Fantastic Fungi ...","Language":"English","Country":"USA","Awards":"N/A","Poster":"https://m.media-amazon.com/images/M/MV5BYmVhZGYwNjgtMzMzYi00ZDAwLTk5YmYtMzVhZDVjMjhiOWQyXkEyXkFqcGdeQXVyMTc3MTI1NzE@._V1_SX300.jpg","Ratings":[],"Metascore":"N/A","imdbRating":"N/A","imdbVotes":"N/A","imdbID":"tt8258074","Type":"movie","DVD":"N/A","BoxOffice":"N/A","Production":"N/A","Website":"N/A","Response":"True"}
row = pd.DataFrame.from_dict(a, orient='index').transpose()
all_movies_df = all_movies_df.append(row)
all_movies_df.shape

(530, 28)

In [347]:
all_movies_df.to_csv('clean_movies_02.csv')

In [298]:
row = pd.DataFrame.from_dict(a, orient='index').transpose()
row

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,Awards,Poster,Ratings,Metascore,imdbRating,imdbVotes,imdbID,Type,totalSeasons,Response
0,KonoSuba - God's Blessing on This Wonderful Wo...,2016–,TV-14,12 Jan 2016,24 min,"Animation, Adventure, Comedy, Fantasy",N/A,N/A,"Jun Fukushima, Sora Amamiya, Rie Takahashi, Ai...",It was a happy day for Kazuma - right up to th...,...,N/A,https://m.media-amazon.com/images/M/MV5BNDNiOW...,"[{'Source': 'Internet Movie Database', 'Value'...",N/A,7.8,"3,332",tt5370118,series,2,True


In [300]:
all_movies_df = all_movies_df.append(row)

In [304]:
all_movies_df

,Actors,Awards,BoxOffice,Country,DVD,Director,Genre,Language,Metascore,Plot,...,Type,Website,Writer,Year,imdbID,imdbRating,imdbVotes,index,level_0,totalSeasons
0,"Robert Downey Jr., Chris Evans, Mark Ruffalo, ...",N/A,N/A,USA,30 Jul 2019,"Anthony Russo, Joe Russo","Action, Adventure, Drama, Sci-Fi","English, Japanese, Xhosa",78,After the devastating events of Avengers: Infi...,...,movie,N/A,"Christopher Markus (screenplay by), Stephen Mc...",2019,tt4154796,8.5,"589,503",0.0,0.0,NaN
1,"Rowan Atkinson, Matthew Broderick, Niketa Cala...",Won 2 Oscars. Another 33 wins & 29 nominations.,"$94,240,635",USA,07 Oct 2003,"Roger Allers, Rob Minkoff","Animation, Adventure, Drama, Family, Musical","English, Swahili, Xhosa, Zulu",88,A Lion cub crown prince is tricked by a treach...,...,movie,N/A,"Irene Mecchi (screenplay by), Jonathan Roberts...",1994,tt0110357,8.5,"869,323",0.0,1.0,NaN
2,"Tom Holland, Samuel L. Jackson, Jake Gyllenhaa...",N/A,N/A,USA,17 Sep 2019,Jon Watts,"Action, Adventure, Sci-Fi","Italian, Czech, English",69,Following the events of Avengers: Endgame (201...,...,movie,N/A,"Chris McKenna, Erik Sommers, Stan Lee (based o...",2019,tt6320628,7.6,"221,516",0.0,2.0,NaN
3,"Brie Larson, Samuel L. Jackson, Ben Mendelsohn...",N/A,N/A,"USA, Australia",11 Jun 2019,"Anna Boden, Ryan Fleck","Action, Adventure, Sci-Fi",English,64,Carol Danvers becomes one of the universe's mo...,...,movie,N/A,"Anna Boden (screenplay by), Ryan Fleck (screen...",2019,tt4154664,7.0,"368,679",0.0,3.0,NaN
4,"Tom Hanks, Tim Allen, Annie Potts, Tony Hale",N/A,N/A,USA,01 Oct 2019,Josh Cooley,"Animation, Adventure, Comedy, Family, Fantasy",English,84,"When a new toy called ""Forky"" joins Woody and ...",...,movie,N/A,"John Lasseter (original story by), Andrew Stan...",2019,tt1979376,8.0,"121,373",0.0,4.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,"Jun Fukuyama, Yukana Nogami, Takahiro Sakurai,...",N/A,N/A,Japan,N/A,Gorô Taniguchi,"Animation, Action, Adventure, Drama, Mystery, ...",Japanese,N/A,The story takes place several years after Lelo...,...,movie,N/A,"J. Michael Tatum (adr script), Ichirô Ôkouchi",2019,tt6344664,7.7,422,NaN,NaN,NaN
0,"Jun Fukuyama, Yukana Nogami, Takahiro Sakurai,...",N/A,N/A,Japan,N/A,Gorô Taniguchi,"Animation, Action, Adventure, Drama, Mystery, ...",Japanese,N/A,The story takes place several years after Lelo...,...,movie,N/A,"J. Michael Tatum (adr script), Ichirô Ôkouchi",2019,tt6344664,7.7,422,NaN,NaN,NaN
0,"Hisaaki Takeuchi, Fumika Shimizu, Tamao Satô, ...",N/A,N/A,Japan,N/A,Hiroshi Akabane,Drama,Japanese,N/A,A man's near death experience inspires him to ...,...,movie,N/A,"Ryuho Okawa (original story by), Sayaka Okawa",2019,tt10572668,N/A,N/A,NaN,NaN,NaN
0,Phoebe Waller-Bridge,N/A,N/A,UK,N/A,N/A,Comedy,N/A,N/A,N/A,...,movie,N/A,Phoebe Waller-Bridge,2019,tt10702760,N/A,N/A,NaN,NaN,NaN


In [255]:
for i in error_list:
    print(df['Release Group'][i-1])

Me and My Motherland
Pain and Glory
Countdown
Fate/stay night [Heaven's Feel] II. lost butterfly
Code Geass: Lelouch of the Re;Surrection 2019 Re-release
Immortal Hero
National Theatre Live: Fleabag
KonoSuba - God's Blessing on This Wonderful World!: Legend of Crimson
Uri: The Surgical Strike 2019 Re-release
Arctic 2019 Re-release
The Oscar Nominated Short Films 2019: Documentary
Spirits in the Forest
Sound! Euphonium the Movie - Our Promise: A Brand New Day
Gone with the Wind 2019 Re-release
The Wizard of Oz 2019 Re-release
Alien 2019 Re-release
BTS World Tour 'Love Yourself: Speak Yourself' (The Final) Seoul Live Viewing
Los Domirriqueños 2
My Neighbor Totoro 2019 Re-release
Jesus Is King
Noah: The Musical
Kiki's Delivery Service 2019 Re-release
Howl's Moving Castle 2019 Re-release
Steel Magnolias 2019 Re-release
Chonda Pierce: Unashamed
Fantastic Fungi
Patterns of Evidence: Moses Controversy
I Love Lucy: A Colorized Celebration
Where's My Roy Cohn?
Dirty Dancing 2019 Re-release
The 

In [253]:
len(error_list)

139

In [220]:
df.columns

Index(['Rank', 'Release Group', 'Worldwide', 'Domestic', '%', 'Foreign',
       '%.1'],
      dtype='object')

In [221]:
all_movies_df.to_csv('clean_movies_01.csv')

In [222]:
a=pd.read_csv('clean_movies_01.csv')

In [223]:
a

,Unnamed: 0,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,...,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response,totalSeasons
0,0,Avengers: Endgame,2019,PG-13,26 Apr 2019,181 min,"Action, Adventure, Drama, Sci-Fi","Anthony Russo, Joe Russo","Christopher Markus (screenplay by), Stephen Mc...","Robert Downey Jr., Chris Evans, Mark Ruffalo, ...",...,8.5,"589,503",tt4154796,movie,30 Jul 2019,NaN,Marvel Studios,NaN,True,NaN
1,0,The Lion King,1994,G,24 Jun 1994,88 min,"Animation, Adventure, Drama, Family, Musical","Roger Allers, Rob Minkoff","Irene Mecchi (screenplay by), Jonathan Roberts...","Rowan Atkinson, Matthew Broderick, Niketa Cala...",...,8.5,"869,323",tt0110357,movie,07 Oct 2003,"$94,240,635",Buena Vista,NaN,True,NaN
2,0,Spider-Man: Far from Home,2019,PG-13,02 Jul 2019,129 min,"Action, Adventure, Sci-Fi",Jon Watts,"Chris McKenna, Erik Sommers, Stan Lee (based o...","Tom Holland, Samuel L. Jackson, Jake Gyllenhaa...",...,7.6,"221,516",tt6320628,movie,17 Sep 2019,NaN,Sony Pictures,NaN,True,NaN
3,0,Captain Marvel,2019,PG-13,08 Mar 2019,123 min,"Action, Adventure, Sci-Fi","Anna Boden, Ryan Fleck","Anna Boden (screenplay by), Ryan Fleck (screen...","Brie Larson, Samuel L. Jackson, Ben Mendelsohn...",...,7.0,"368,679",tt4154664,movie,11 Jun 2019,NaN,Marvel Studios,NaN,True,NaN
4,0,Toy Story 4,2019,G,21 Jun 2019,100 min,"Animation, Adventure, Comedy, Family, Fantasy",Josh Cooley,"John Lasseter (original story by), Andrew Stan...","Tom Hanks, Tim Allen, Annie Potts, Tony Hale",...,8.0,"121,373",tt1979376,movie,01 Oct 2019,NaN,Disney/Pixar,NaN,True,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,0,Jimi Hendrix Electric Church,2015,Not Rated,15 Jan 2017,89 min,Documentary,John McDermott,NaN,"Jimi Hendrix, Bob Merlis, Billy Cox, Susan Ted...",...,7.5,154,tt5217608,movie,NaN,NaN,Abramorama,NaN,True,NaN
500,0,Game Day,1999,R,21 Sep 1999,106 min,"Comedy, Drama, Sport",Steve Klein,Steve Klein,"Richard Lewis, Sean Squire, Gerry Becker, Alic...",...,5.5,203,tt0146786,movie,18 Dec 2001,NaN,NaN,NaN,True,NaN
501,0,The Hours and Times,1991,NaN,01 Jan 1992,57 min,"Drama, Music",Christopher Munch,Christopher Munch,"David Angus, Ian Hart, Stephanie Pack, Robin M...",...,6.3,562,tt0104448,movie,31 May 2005,NaN,Oscilloscope Laboratories,NaN,True,NaN
502,0,The Untold Story,2019,NaN,11 Jan 2019,104 min,"Comedy, Drama",Shane Stanley,"Lee Stanley, Shane Stanley","Miko Hughes, Nia Peeples, Ellen Greene, Jordan...",...,5.7,216,tt3737840,movie,NaN,NaN,Ammo Content,NaN,True,NaN


In [267]:
all_movies_df.columns

Index(['Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director',
       'Writer', 'Actors', 'Plot', 'Language', 'Country', 'Awards', 'Poster',
       'Ratings', 'Metascore', 'imdbRating', 'imdbVotes', 'imdbID', 'Type',
       'DVD', 'BoxOffice', 'Production', 'Website', 'Response',
       'totalSeasons'],
      dtype='object')